In [1]:
import gzip, pickle
f = gzip.open("data/full_process_by_day.pkl.gz")
realizations = pickle.load(f)
f.close()

In [2]:
def reset_time(realization):
    min_time = min(process[0] for process in realization if len(process) > 0)
    for process in realization:
        for i in range(len(process)):
            process[i] -= min_time

import numpy as np
ticks = []
for day in range(len(realizations)):
    ticks_day = []
    for dim in range(len(realizations[0])):
        data_plus_noise = realizations[day][dim] + np.random.rand(len(realizations[day][dim]))
        ticks_day.append(data_plus_noise)
    reset_time(ticks_day)
    if all(len(process) > 0 for process in realizations[day]):
        ticks.append(ticks_day)

# NPHC

In [3]:
from nphc.main import NPHC
nphc = NPHC()
nphc.fit(ticks,half_width=3600,filtr="rectangular",method="parallel_by_day")

L is computed


/Users/massil/anaconda/envs/py3k/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


C is computed
K_c is computed


In [4]:
R_pred = nphc.solve(training_epochs=200,display_step=200,learning_rate=1e-2,optimizer='adam')
d = R_pred.shape[0]
from scipy.linalg import inv
import numpy as np
G_nphc = np.eye(d) - inv(R_pred)

Epoch: 0000 log10(cost)= 0.044227865
Optimization Finished!


/Users/massil/Programmation/git/nphc/nphc/main.py:132: ComplexWarning: Casting complex values to real discards the imaginary part
  R0 = tf.constant(start_point.astype(np.float32), shape=[d,d])


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#sns.heatmap(G_nphc[:20,:20],cmap="Blues")

# ADM4

In [ ]:
from mlpp.hawkesem import HawkesEMADM4, plot_hawkes_kernels

In [ ]:
d = len(realizations[0])
beta = 1./3600
rho = 1e0
lbd1 = 0.
lbd2 = 0.
mu = np.zeros(d)+.2
auv = np.zeros((d,d))+.2
em = HawkesEMADM4(beta,rho,lbd1,lbd2,n_cores=8)
em.fit(ticks)
em.solve(niter=10,mu_start=mu,auv_start=auv,flag_verbose=False)